In [ ]:
-- select * from  dsc_dwd.dwd_wh_dsc_inventory_dtl_di 
-- where src = 'scale'
-- and data_source in ('scale_hpi', 'scale_michelin', 'scale_fas', 'scale_bose')
-- and inc_day in ('20211124', '20211117', '20211110', '20211103', 
-- '20211027', '20211020', '20211013','20211006')
-- and usage_flag = '1'

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import re
import os
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

print(os.listdir('data_down'))

['inv_7.csv']


In [2]:
%%time
df = pd.read_csv('./data_down/inv_7.csv', sep='\001')
df.columns = [re.sub('\w+\.', '', i) for i in list(df.columns)]
df = df.dropna(how = 'all', axis =1)
time_cols = pd.Series(df.columns)[pd.Series(df.columns).str.lower().str.findall('date|time').apply(len)>0]
df[time_cols] = df[time_cols].apply(lambda x: x.str.slice(0,10))
df9 = df 

Wall time: 9.71 s


In [164]:
def load_data(data_source):
    """
    load bose data;
    所有类型的qty都要加起来哦
    只选择有多个收货日期的货物
    """
    df = df9
    df = df[df['data_source'] == data_source]
    df ['recived_date'] = pd.to_datetime(df['recived_date'])
    df = df[['wms_company_name', 'wms_warehouse_id','sku_code', 'sku_name', 'sku_desc', 'location_zone',\
        'lock_codes', 'on_hand_qty', 'in_transit_qty','allocated_qty', 'shelf_days', 
        'recived_date','usage_flag', 'inc_day']]
    qty = pd.Series(df.columns).str.extract('(.+qty)').dropna(axis = 0)
    df['qty'] = df[qty[0]].sum(axis = 1).round(2)
    # bose_inv.drop_duplicates(subset = ['sku_code', 'recived_date', 'lock_codes', 'inc_day', 'qty'])
    # 没有重复的 目前看....aaa
    df = df.groupby(['recived_date', 'sku_code', 'lock_codes','inc_day', 'wms_warehouse_id'], dropna = False).agg(
    {
        'qty':sum,
        'location_zone': set
    }
    ).sort_values(['sku_code', 'recived_date']).reset_index()
    # 只选择有多个收货日期的货物
    filter0 = df.groupby(['sku_code'])['recived_date'].agg(
    {
        set
    }
        ).reset_index()

    filter0 = pd.DataFrame(filter0[filter0['set'].apply(len)> 1]['sku_code'].drop_duplicates())
    bose_inv = filter0.merge(df, on = ['sku_code'], how = 'inner')\
        .sort_values(['recived_date','sku_code', 'inc_day'])
    
    return bose_inv

bose_inv = load_data('scale_bose')

In [165]:
# # 只选择多次出库的货物
# filter = bose.groupby(['recived_date', 'sku_code'])['inc_day'].agg({set}).reset_index()
# filter = filter[filter['set'].apply(len)> 1][['recived_date','sku_code']].drop_duplicates();filter
bose_inv.head(4)

,sku_code,recived_date,lock_codes,inc_day,wms_warehouse_id,qty,location_zone
13036,722713-0010,2015-09-08,1000,20211006,BOSE_SH,13.0,{L-CE-1000}
13037,722713-0010,2015-09-08,1000,20211013,BOSE_SH,13.0,{L-CE-1000}
13038,722713-0010,2015-09-08,1000,20211020,BOSE_SH,13.0,{L-CE-1000}
13039,722713-0010,2015-09-08,1000,20211027,BOSE_SH,13.0,{L-CE-1000}


In [166]:
%%time
df0 = pd.DataFrame()
def snapshot():
    """
    pivot table. inc_day 快照 作为 cols
    添加标记.
    """
    global df0, bose_inv
    for i in tqdm(bose_inv['sku_code'].unique(), ascii= False, colour = 'green'):
        df_out = bose_inv[bose_inv['sku_code'] == i]\
            .pivot_table(columns=['inc_day'], index = 'recived_date', values=['qty']).reset_index()
        df_out['sku_code'] = i
        df0 = pd.concat([df0, df_out], axis = 0)
    try:
        df0.columns = df0.columns.get_level_values(level=1)
    except:
        pass
    df0.columns = list(df0.columns[0:8]) + ['received_date','sku']
    df0 = df0.sort_values(['sku', 'received_date'])
    df0['mark'] = 0
    # 这个吊东`西不知道为什么是反选的 
    df0['mark'] = df0['mark'].where(df0.iloc[:, 0:7].isna().all(axis = 1) == False, 'new')
    df0['mark'] = df0['mark'].where(~df0.iloc[:,7].isna() , 'clear')
    bose_inv = bose_inv.rename({'sku_code':'sku', 'recived_date':'received_date'}, axis = 1)\
        .reset_index(drop = True).drop(['inc_day', 'qty'], axis = 1)
    bose_inv = bose_inv.drop_duplicates(subset = ['sku', 'received_date', 'lock_codes'])
    df0 = df0.merge(bose_inv, on = ['sku','received_date'], how = 'left')   
snapshot()

100%|██████████| 1034/1034 [00:49<00:00, 20.76it/s]

Wall time: 49.9 s


In [167]:
def err_part():
    """
    findout who are the naught peach.
    err 中干掉了 new 干掉了maylock 
    """
    df_err = df0[df0['mark'] != 'new']
    # 补充可能被锁的标记
    df_err['mark'] = df_err['mark'].where(df_err.iloc[:, 0:8].fillna(0).nunique(axis = 1) > 1, 'may_lock')
    # 干掉了maylock
    df_err = df_err[df_err['mark'] != 'may_lock'].sort_values(['sku', 'received_date'])
    df_err['change'] = df_err.iloc[:,0:8].diff(axis = 1).sum(axis = 1)
    shift = df_err.groupby('sku').shift(1) 
    # q = []
    # for i in shift['mark']:
    #     if pd.isna(i):
    #         q.append('clear')
    #     elif i =='clear':
    #         q.append('clear')
    #     else:
    #         q.append('check')
    # shift['mark'] = q
    # shift['mark2'] = shift.iloc[:,0:8].diff(axis = 1).sum(axis = 1)
    shift = shift[['mark','change']]
    shift.columns = ['lag_mark', 'lag_change']
    shift['lag_mark'] = shift['lag_mark'].where(~shift['lag_mark'].isna(), 'clear')
    df_err = pd.concat([df_err, shift], axis = 1)
    # df_wrong = df_err[(df_err['lag_mark'] != 'clear') & (df_err['change'] != 0)]
    # df_good  = df_err[(df_err['lag_mark'] != 'clear') | (df_err['change'] != 0)]
    return df_err
df_err = err_part()
    # shift['mark'] = ['clear' if (i == np.nan) 'clear'  elif (i =='clear') else 'check' for i in shift['mark']];shift

In [168]:
# df_err.to_pickle('./data_up/df_err_bose.p')
# df0.to_pickle('./data_up/df0_bose.p')
df_err_bose = pd.read_pickle('./data_up/df_err_bose.p')
df0_bose = pd.read_pickle('./data_up/df0_bose.p')


In [169]:

def output(df_err, df0):
    dishes = list(df_err[(df_err['lag_mark'] != 'clear') & (df_err['change'] != 0)]['sku'].unique())
    return df0[df0['sku'].isin(dishes)]
def check(sku, df0):
    return df0[(df0['lock_codes'] == '1000') & (df0['sku'] == sku)]


In [170]:
output(df_err_bose,df0_bose)
# 1000全新 3000退货 4000破损产品

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
3,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,2020-03-06,000000000000024643,0,4000,BOSE_SH,{L-CE-4000}
4,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,2020-04-21,000000000000024643,0,4000,BOSE_SH,{L-CE-4000}
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2021-04-14,000000000000024643,0,4000,BOSE_SH,{L-CE-1000}
6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-04,000000000000024643,clear,1000,BOSE_SH,{L-CE-1000}
7,NaN,NaN,98.0,98.0,98.0,97.0,95.0,95.0,2021-10-15,000000000000024643,0,1000,BOSE_SH_Bonded,{L-CE-1000}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6751,847.0,460.0,169.0,NaN,NaN,NaN,NaN,NaN,2021-09-24,866724-0200,clear,1000,BOSE_SH,{L-CE-1000}
6752,768.0,768.0,768.0,NaN,NaN,NaN,NaN,NaN,2021-09-26,866724-0200,clear,1000,BOSE_SH,{L-CE-1000}
6753,NaN,NaN,768.0,NaN,NaN,NaN,NaN,NaN,2021-10-20,866724-0200,clear,1000,BOSE_SH_Bonded,{L-CE-1000}
6754,NaN,NaN,NaN,NaN,1152.0,231.0,231.0,231.0,2021-11-03,866724-0200,0,1000,BOSE_SH_Bonded,{L-CE-1000}


In [133]:
output(df_err_bose,df0_bose)['sku'].unique()

In [171]:
check('866724-0200', df0_bose)

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
6751,847.0,460.0,169.0,NaN,NaN,NaN,NaN,NaN,2021-09-24,866724-0200,clear,1000,BOSE_SH,{L-CE-1000}
6752,768.0,768.0,768.0,NaN,NaN,NaN,NaN,NaN,2021-09-26,866724-0200,clear,1000,BOSE_SH,{L-CE-1000}
6753,NaN,NaN,768.0,NaN,NaN,NaN,NaN,NaN,2021-10-20,866724-0200,clear,1000,BOSE_SH_Bonded,{L-CE-1000}
6754,NaN,NaN,NaN,NaN,1152.0,231.0,231.0,231.0,2021-11-03,866724-0200,0,1000,BOSE_SH_Bonded,{L-CE-1000}
6755,NaN,NaN,NaN,NaN,NaN,278.0,217.0,47.0,2021-11-09,866724-0200,0,1000,BOSE_SH,{L-CE-1000}


=================================
# HP
=================================


In [172]:
%%time
del df0 ,bose_inv
bose_inv = load_data('scale_hpi');bose_inv
df0 = pd.DataFrame()
snapshot()

# snapshot()
df_err = err_part()

100%|██████████| 1483/1483 [01:15<00:00, 19.64it/s]


Wall time: 1min 22s


In [173]:
# df_err.to_pickle('./data_up/df_err_hp.p')
# df0.to_pickle('./data_up/df0_hp.p')
df_err_hp = pd.read_pickle('./data_up/df_err_hp.p')
df0_hp = pd.read_pickle('./data_up/df0_hp.p')


In [174]:
output(df_err_hp, df0_hp)

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
41,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-13,1VV21AA,clear,1004-34,HPI_SH,{L_Normal}
42,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-19,1VV21AA,clear,1004-34,HPI_SH,{L_Normal}
43,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-22,1VV21AA,clear,1004-34,HPI_SH,{L_Normal}
44,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-05,1VV21AA,clear,1004-34,HPI_SH,{L_Normal}
45,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-09,1VV21AA,clear,1004-34,HPI_SH,{L_Normal}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12138,10.0,10.0,4.0,4.0,4.0,4.0,4.0,NaN,2021-09-29,Z7Y79A,clear,1001-34,HPI_WH,{L_Normal}
12188,36.0,18.0,6.0,6.0,6.0,6.0,6.0,6.0,2021-08-31,Z9M07A,0,1001-34,HPI_WH,{L_Normal}
12189,36.0,18.0,6.0,6.0,6.0,6.0,6.0,6.0,2021-08-31,Z9M07A,0,1002-34,HPI_WH,{L_Normal}
12190,36.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-27,Z9M07A,clear,1001-34,HPI_WH,{L_Normal}


In [180]:
# output(df_err_hp, df0_hp)['sku'].unique()

array(['1VV21AA', '1VV22AA', '1XA55A', '1XA81A', '22A80A', '28B51A#AB2',
       '2EH31A', '2EZ26A', '2PD46A', '2RM82A#BCD', '3ED78A', '3GX99A#AB2',
       '3JA77AA', '3JA78AA', '3JA79AA', '3JA80AA', '3JA81AA', '3JA82AA',
       '3JA83AA', '3JA84AA', '3JB06AA', '3JN69A', '3PZ35A#AB2',
       '3QA35A#AB2', '3UK97D#AB2', '3WT89MC', '3WX02A', '3WX07A',
       '3WX08A', '3WX30A', '3YM15AA', '3YM17AA', '3YM19AA', '3YM20AA',
       '3YM21AA', '3YM22AA', '3YM70AA#AB2', '3YM71AA#AB2', '3YM72AA#AB2',
       '3YM73AA#AB2', '3YM76AA#AB2', '3YM77AA#AB2', '3YN07AA', '3YN09AA',
       '3YP42AA#AB2', '3YP93AA', '3YP94AA', '3YW73A#AB2', '4KJ64D#AB2',
       '4PZ47A#AB2', '4VW13A#BCD', '4YE53A#AB2', '4YE56A#AB2', '4YL17MC',
       '4ZA22A', '4ZA23A', '4ZA24A', '4ZB81A#AB2', '51645AA#007',
       '5NL12A#AB2', '5RC84A#AB2', '5RC86A#AB2', '5RC87A#AB2',
       '5SD79B#AB2', '5VA74A#AB2', '6GX04A#AB2', '6GX05A#AB2',
       '6GX06A#AB2', '6ZD21AA#AB2', '6ZD29AA', '6ZD30AA', '7FR55D#AB2',
       '7FR60D#AB2',

In [175]:
def check(sku, df0):
    return df0[df0['sku'] == sku]


In [181]:
check('3YM15AA', df0_hp)

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
600,324.5,394.0,276.5,308.0,407.0,341.0,137.0,116.5,2021-08-25,3YM15AA,0,1001-34,HPI_SH,{L_Normal}
601,324.5,394.0,276.5,308.0,407.0,341.0,137.0,116.5,2021-08-25,3YM15AA,0,1008-34,HPI_SH,{L_Normal}
602,324.5,394.0,276.5,308.0,407.0,341.0,137.0,116.5,2021-08-25,3YM15AA,0,1008-27,HPI_SH,{L_Normal}
603,120.0,120.0,120.0,130.0,55.0,55.0,55.0,55.0,2021-09-02,3YM15AA,0,1001-34,HPI_SH,{L_Normal}
604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,840.0,2021-11-19,3YM15AA,new,1001-34,HPI_SH,{L_Normal}


=================================
# michelin
=================================


In [182]:
%%time
del df0 ,bose_inv
bose_inv = load_data('scale_michelin');bose_inv
df0 = pd.DataFrame()
snapshot()

# snapshot()
df_err = err_part()

100%|██████████| 1360/1360 [01:24<00:00, 16.02it/s]


Wall time: 1min 31s


In [183]:
# df_err.to_pickle('./data_up/df_err_mich.p')
# df0.to_pickle('./data_up/df0_mich.p')
df_err_mich = pd.read_pickle('./data_up/df_err_mich.p')
df0_mich = pd.read_pickle('./data_up/df0_mich.p')


In [184]:
output(df0=df0_mich, df_err=df_err_mich)

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
0,15.0,14.0,18.0,10.0,10.0,10.0,10.0,10.0,2021-09-14,000219_111,0,Available,RT,{L_label}
1,96.0,96.0,96.0,96.0,96.0,96.0,96.0,96.0,2021-09-20,000219_111,0,Available,RT,{L_label}
2,NaN,NaN,25.0,25.0,21.0,21.0,21.0,21.0,2021-10-15,000219_111,0,RETURN_FR-1F3,RT,{nan}
11,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2020-08-09,001068_109,0,RETURN_IM-1IM,RT,{nan}
12,2.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,2021-08-08,001068_109,clear,Available,RT,{L_label}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11503,16.0,13.0,13.0,13.0,13.0,13.0,12.0,4.0,2021-07-03,MATMICHELIN00591,0,Available,NaN,{nan}
11504,30.0,14.0,11.0,87.0,87.0,87.0,91.0,91.0,2021-09-02,MATMICHELIN00591,0,Available,NaN,{nan}
11505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2021-11-21,MATMICHELIN00591,new,RETURN_IM-1IM,NaN,{nan}
11506,16.0,4.0,3.0,79.0,79.0,77.0,72.0,73.0,2021-07-03,MATMICHELIN00592,0,Available,NaN,{nan}


In [191]:
def check(sku, df0):
    return df0[(df0['lock_codes'] == 'Available')& (df0['sku'] == sku)]

check('001068_109',df0 = df0_mich)

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
12,2.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,2021-08-08,001068_109,clear,Available,RT,{L_label}
14,NaN,NaN,NaN,89.0,2.0,6.0,NaN,NaN,2021-10-21,001068_109,clear,Available,RT,"{nan, L_label}"
16,NaN,NaN,NaN,NaN,50.0,50.0,64.0,69.0,2021-10-31,001068_109,0,Available,RT,{L_label}
17,NaN,NaN,NaN,NaN,2.0,2.0,1.0,1.0,2021-11-02,001068_109,0,Available,RT,{L_label}
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.0,2021-11-20,001068_109,new,Available,RT,{L_label}
